import modules

In [188]:
import pandas as pd
from datetime import date, timedelta
import datetime
import holidays
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: c:\apps\Project1\mid_code


load data

In [189]:
cny = pd.read_csv('../dataset/CNY_KRW.csv')[['날짜','종가','시가']].rename(
    columns={'날짜':'date',
             '종가':'close',
             '시가':'open'})

In [190]:
cny.sort_values(by='date', inplace=True)
print(cny.info(), cny.head(), cny.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 66 entries, 65 to 0
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    66 non-null     object 
 1   close   66 non-null     float64
 2   open    66 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.1+ KB
None             date   close    open
65  2023- 07- 28  178.02  179.00
64  2023- 07- 31  178.90  178.10
63  2023- 08- 01  179.58  178.96
62  2023- 08- 02  180.60  179.64
61  2023- 08- 03  181.24  180.69            date   close    open
4  2023- 10- 23  183.45  184.57
3  2023- 10- 24  183.82  183.66
2  2023- 10- 25  184.81  183.90
1  2023- 10- 26  184.88  184.89
0  2023- 10- 27  184.98  184.72


In [191]:
# 한국의 공휴일 가져오기
kr_holidays = holidays.SouthKorea(years=2023)

# 기간 설정 (예: 2023년 1월 1일부터 12월 31일까지)
start_date = date(2023, 7, 28)
end_date = date(2023, 10, 20)

# 한국의 공휴일을 제외한 날짜 리스트 생성
date_list = []
current_date = start_date
while current_date <= end_date:
    if current_date in kr_holidays:
        date_list.append(current_date)
    current_date += timedelta(days=1)

In [192]:
date_list

[datetime.date(2023, 8, 15),
 datetime.date(2023, 9, 28),
 datetime.date(2023, 9, 29),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 10, 3),
 datetime.date(2023, 10, 9)]

In [193]:
cny['date'] = pd.to_datetime(cny['date'])

In [194]:
cny = cny.loc[~cny['date'].isin(date_list)]

In [195]:
print(cny.info(), cny.head(), cny.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 65 to 0
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    61 non-null     datetime64[ns]
 1   close   61 non-null     float64       
 2   open    61 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.9 KB
None          date   close    open
65 2023-07-28  178.02  179.00
64 2023-07-31  178.90  178.10
63 2023-08-01  179.58  178.96
62 2023-08-02  180.60  179.64
61 2023-08-03  181.24  180.69         date   close    open
4 2023-10-23  183.45  184.57
3 2023-10-24  183.82  183.66
2 2023-10-25  184.81  183.90
1 2023-10-26  184.88  184.89
0 2023-10-27  184.98  184.72


#### 환율 변동값 전처리
* 전일 변동율 = (전날종가-전날시가)/전날시가 yesterday_change <br>
* 야간 변동율 = (당일시가-전날종가)/전날종가 night_change <br>
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [196]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
cny['cny_yesterday_change'] = round((cny['close']-cny['open'])/cny['open']*100, 2)
cny['cny_yesterday_change'] = cny['cny_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
cny['close_shift'] = cny['close'].shift(1)
cny['cny_night_change'] = round((cny['open']-cny['close_shift'])/cny['close_shift']*100,2)

cny.drop('close_shift',axis=1,inplace=True)
print(cny.info(), cny.head(), cny.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 65 to 0
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  61 non-null     datetime64[ns]
 1   close                 61 non-null     float64       
 2   open                  61 non-null     float64       
 3   cny_yesterday_change  60 non-null     float64       
 4   cny_night_change      60 non-null     float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 2.9 KB
None          date   close    open  cny_yesterday_change  cny_night_change
65 2023-07-28  178.02  179.00                   NaN               NaN
64 2023-07-31  178.90  178.10                 -0.55              0.04
63 2023-08-01  179.58  178.96                  0.45              0.03
62 2023-08-02  180.60  179.64                  0.35              0.03
61 2023-08-03  181.24  180.69                  0.53              0.05         date 

In [197]:
# 데이터 기간 : 2023/07/31 ~ 2023/10/20
cny = cny.loc[cny['date']<='2023-10-20',].loc[cny['date']>='2023-07-31',]
cny = cny[['date','cny_yesterday_change','cny_night_change']]
print(cny.info(), cny.head(), cny.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 55 entries, 64 to 5
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  55 non-null     datetime64[ns]
 1   cny_yesterday_change  55 non-null     float64       
 2   cny_night_change      55 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.7 KB
None          date  cny_yesterday_change  cny_night_change
64 2023-07-31                 -0.55              0.04
63 2023-08-01                  0.45              0.03
62 2023-08-02                  0.35              0.03
61 2023-08-03                  0.53              0.05
60 2023-08-04                  0.30              0.09         date  cny_yesterday_change  cny_night_change
9 2023-10-16                  0.15              0.08
8 2023-10-17                 -0.46              0.03
7 2023-10-18                  0.30              0.08
6 2023-10-19 

In [198]:
# 날짜순으로 오름차순 정렬
cny.sort_values(by='date', inplace=True)
cny.reset_index(drop=True, inplace=True)

In [199]:
print(cny.info(), cny.head(), cny.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  55 non-null     datetime64[ns]
 1   cny_yesterday_change  55 non-null     float64       
 2   cny_night_change      55 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.4 KB
None         date  cny_yesterday_change  cny_night_change
0 2023-07-31                 -0.55              0.04
1 2023-08-01                  0.45              0.03
2 2023-08-02                  0.35              0.03
3 2023-08-03                  0.53              0.05
4 2023-08-04                  0.30              0.09          date  cny_yesterday_change  cny_night_change
50 2023-10-16                  0.15              0.08
51 2023-10-17                 -0.46              0.03
52 2023-10-18                  0.30              0.08
53 2023-10

csv 파일로 새로 저장

In [200]:
cny.to_csv("../dataset/clean_feature/cny_krw_final.csv")